In [ ]:
import os

import pandas as pd

from utils import load_raw_surface_data
from utils import load_raw_volume_data

# Surface

In [ ]:
# remove rows from all files that are corresponding to the one missing in
# DipoleVertical_Im_Ey_d5mm.txt and DipoleHorizontal_Re_Hx_d15mm.txt files

path_raw = os.path.join('data', 'raw', 'surface')
antennas = ['DipoleVertical', 'DipoleVertical', 'DipoleHorizontal']
distances = [5, 15, 15]
components = ['Im_Ey', 'Im_Hy', 'Re_Hx']
critical_idxs = []

for antenna, distance, component in zip(antennas, distances, components):
    # load the problematic data
    df_1 = pd.read_csv(  # take this file as the reference
        os.path.join(path_raw, f'{antenna}_Re_Ex_d{distance}mm.txt'),
        sep='\s+', comment='%', header=None,
        usecols=[0, 1, 2], names=['x', 'y', 'z']
    )
    df_2 = pd.read_csv(
        os.path.join(path_raw, f'{antenna}_{component}_d{distance}mm.txt'),
        sep='\s+', comment='%', header=None,
        usecols=[0, 1, 2], names=['x', 'y', 'z']
    )

    # find where the files differ
    for idx, (row_1, row_2) in enumerate(zip(df_1.values[:-1], df_2.values)):
        diff = sum(row_1 - row_2)
        if diff != 0.0:
            critical_idxs.append(idx)
            break

In [ ]:
critical_idxs

In [ ]:
# create a clean dataset
path_clean = os.path.join('data', 'clean', 'surface')
antennas = ['DipoleVertical', 'DipoleHorizontal',
            'ArrayVertical', 'ArrayHorizontal']
distances = [5, 10, 15]

for antenna in antennas:
    for distance in distances:
        if (antenna == 'DipoleVertical') and (distance == 5):
            critical_idx = critical_idxs[0]
        elif (antenna == 'DipoleVertical') and (distance == 15):
            critical_idx = critical_idxs[1]
        elif antenna == 'DipoleHorizontal':
            critical_idx = critical_idxs[2]
        else:
            critical_idx = None
        xyz, (Ex, Ey, Ez), (Hx, Hy, Hz) = load_raw_surface_data(antenna,
                                                                distance,
                                                                critical_idx)
        df = pd.DataFrame(data=xyz)
        df = pd.concat([df,
                        Ex.rename('Ex').astype('complex128'), Ey.rename('Ey'), Ez.rename('Ez'),
                        Hx.rename('Hx'), Hy.rename('Hy'), Hz.rename('Hz'),
                        ], axis=1)
        # df.to_csv(os.path.join(path_clean, f'{antenna}_d{distance}mm.csv'))

In [ ]:
df

# Volume

In [ ]:
# tba